# Multiverse Math

In this task, the agent is operating in an alternate universe which in which the basic mathematical operations like addition and multiplication are different.

The agent must use tools that allow is to carry out calculations in this universe.

This task can help verify that an agent is able to ignore its own knowledge of math and instead correctly use information returned by the tools.

The modified mathematical operations yield different reuslts, but still retain some properties (e.g., the modified multiplication operation is still commutative).

Please note that the modified operations are not guaranteed to even make sense in the real world since not all properties will be retained (e.g., distributive property).

------------------

For this code to work, please configure LangSmith environment variables with your credentials.

```python
import os

os.environ["LANGCHAIN_API_KEY"] = "ls_.."  # Your LangSmith API key
```

In [1]:
from langchain_benchmarks import clone_public_dataset, registry

In [2]:
task = registry["Multiverse Math"]
task

Name,Multiverse Math
Type,ToolUsageTask
Dataset ID,594f9f60-30a0-49bf-b075-f44beabf546a
Description,"An environment that contains a few basic math operations, but with altered results. For example, multiplication of 5*3 will be re-interpreted as 5*3*1.1. The basic operations retain some basic properties, such as commutativity, associativity, and distributivity; however, the results are different than expected. The objective of this task is to evaluate the ability to use the provided tools to solve simple math questions and ignore any innate knowledge about math."


Clone the dataset associated with this task

In [3]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Multiverse Math already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/ddca73f1-ceda-4562-8c49-7ee0a9df2a01.


## The Environment

Let's check the environment

In [4]:
env = task.create_environment()
env.tools[:5]

[StructuredTool(name='multiply', description='multiply(a: float, b: float) -> float - Multiply two numbers; a * b.', args_schema=<class 'pydantic.v1.main.multiplySchemaSchema'>, func=<function multiply at 0x7fbbdc805bc0>),
 StructuredTool(name='add', description='add(a: float, b: float) -> float - Add two numbers; a + b.', args_schema=<class 'pydantic.v1.main.addSchemaSchema'>, func=<function add at 0x7fbbdc805f80>),
 StructuredTool(name='divide', description='divide(a: float, b: float) -> float - Divide two numbers; a / b.', args_schema=<class 'pydantic.v1.main.divideSchemaSchema'>, func=<function divide at 0x7fbbdc805ee0>),
 StructuredTool(name='subtract', description='subtract(a: float, b: float) -> float - Subtract two numbers; a - b.', args_schema=<class 'pydantic.v1.main.subtractSchemaSchema'>, func=<function subtract at 0x7fbbdc806160>),
 StructuredTool(name='power', description='power(a: float, b: float) -> float - Raise a number to a power; a ** b.', args_schema=<class 'pydant

Multiplying 2 x 4 = 8.8!!

In [5]:
env.tools[0].invoke({"a": 2, "b": 4})

8.8

The task instructions

In [6]:
task.instructions

'You are requested to solve math questions in an alternate mathematical universe. The operations have been altered to yield different results than expected. Do not guess the answer or rely on your  innate knowledge of math. Use the provided tools to answer the question. While associativity and commutativity apply, distributivity does not. Answer the question using the fewest possible tools. Only include the numeric response without any clarifications.'

## Agent Factory

For evaluation, we need an agent factory that will create a new instance of an agent executor for every evaluation run.

We'll use an `OpenAIAgentFactory` provided with LangChain Benchmarks -- look at the `intro` section to see how to define your own.

In [7]:
from langchain_benchmarks.tool_usage import agents

agent_factory = agents.OpenAIAgentFactory(task, model="gpt-4-0613")

# Let's test that our agent works
agent = agent_factory.create()
agent.invoke({"question": "how much is 3 + 5"})

{'input': 'how much is 3 + 5',
 'output': '9.2',
 'intermediate_steps': [(AgentActionMessageLog(tool='add', tool_input={'a': 3, 'b': 5}, log="\nInvoking: `add` with `{'a': 3, 'b': 5}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"a": 3,\n"b": 5\n}', 'name': 'add'}})]),
   9.2)]}

## Eval

Let's evaluate an agent now

In [8]:
import uuid
from langsmith.client import Client
from langchain_benchmarks.tool_usage import get_eval_config

experiment_uuid = uuid.uuid4().hex[:4]

client = Client()

models = ["gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-4-0613"]

for model in models:
    eval_config = get_eval_config()
    agent_factory = agents.OpenAIAgentFactory(task, model=model)
    test_run = client.run_on_dataset(
        dataset_name=task.name,
        llm_or_chain_factory=agent_factory,
        evaluation=eval_config,
        verbose=False,
        project_name=f"oai-functions-{model}-{experiment_uuid}",
        tags=[model],
        project_metadata={
            "model": model,
            "arch": "openai-functions-agent",
            "id": experiment_uuid,
        },
    )

View the evaluation results for project 'oai-functions-gpt-3.5-turbo-1106-1ae4' at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/projects/p/4821711a-a0e6-49e8-abd6-03bb6f5b6aac?eval=true

View all tests for Dataset Multiverse Math at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/ddca73f1-ceda-4562-8c49-7ee0a9df2a01
[------------------------------------------------->] 10/10
 Eval quantiles:
       inputs.question outputs.input                        outputs.output  \
count               10            10                                    10   
unique              10            10                                    10   
top        Add 2 and 3   Add 2 and 3  The result of adding 2 and 3 is 6.2.   
freq                 1             1                                     1   
mean               NaN           NaN                                   NaN   
std                NaN           NaN                                   NaN   
min     

## Analyze

You can take a look at the underlying results.

In [11]:
import pandas as pd
from langsmith.client import Client

client = Client()
projects = list(client.list_projects(reference_dataset_name='Multiverse Math'))

dfs = []
for project in projects:
    first_root_run = next(client.list_runs(project_name=project.name, execution_order=1))
    # Temporary way to get tag information
    tags = first_root_run.tags
    test_results = client.get_test_results(project_name=project.name)
    test_results['model'] = tags[0]
    dfs.append(test_results)


df = pd.concat(dfs)

In [13]:
df.head()

,reference.reference,reference.expected_steps,input.question,outputs.input,outputs.output,outputs.intermediate_steps,feedback.# steps / # expected steps,feedback.Intermediate steps correctness,execution_time,error,id,model
example_id,,,,,,,,,,,,
2392a986-43a3-4a8e-9d28-f541d307f95c,6.20,[add],Add 2 and 3,Add 2 and 3,6.2,"[[{'log': ' Invoking: `add` with `{'a': 2, 'b'...",1.0,1.0,7.090434,None,d29821eb-5983-4b71-ad4e-31a6b44958d1,gpt-4-0613
dbc02325-c207-4d6e-8401-d01d022b0603,-4.00,[subtract],Subtract 3 from 2,Subtract 3 from 2,-4,[[{'log': ' Invoking: `subtract` with `{'a': 2...,1.0,1.0,7.114504,None,76759b2a-4444-4087-8152-59b745b48fbe,gpt-4-0613
a9e5144b-db27-4fb0-9794-2126ba180302,-5.00,[negate],What is -5 if evaluated using the negate funct...,What is -5 if evaluated using the negate funct...,5,[[{'log': ' Invoking: `negate` with `{'a': 5}`...,1.0,1.0,5.934506,None,de287d77-3504-4fc7-ab61-58bf42a4a587,gpt-4-0613
0f6b7976-f06a-4a5d-b12b-32d7735296a8,32.00,[power],what is the result of 2 to the power of 3?,what is the result of 2 to the power of 3?,32.0,"[[{'log': ' Invoking: `power` with `{'a': 2, '...",1.0,1.0,5.691349,None,17ee2105-96f0-4513-aa1f-1c6113d3de54,gpt-4-0613
b79383a3-baee-4fe8-8e9c-f3c7044ad146,32.34,"[add, multiply]",I ate 1 apple and 2 oranges every day for 7 da...,I ate 1 apple and 2 oranges every day for 7 da...,32.34,"[[{'log': ' Invoking: `add` with `{'a': 1, 'b'...",1.0,0.0,11.231384,None,04f7ab28-c629-4533-a88d-f8e001544b33,gpt-4-0613


The dataset is too small to say whether there's a substantial difference between the Open AI models.

However, it seems that the current agent is doing only a mediocare job at using tools.

In [14]:
count_df = df1.groupby('model').size().to_frame('n')
df1.groupby('model')[['feedback.Intermediate steps correctness', 'execution_time']].mean().join(count_df)

,feedback.Intermediate steps correctness,execution_time,n
model,,,
gpt-3.5-turbo-0613,0.7,8.058960,10
gpt-3.5-turbo-1106,0.6,8.209020,10
gpt-4-0613,0.6,9.285144,10
